In [1]:
import os
import os.path as path
import gc
import re
import math
import json
import random

In [2]:
from datetime import datetime
import time

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_row', 50)
pd.set_option('display.max_columns', 100)

In [4]:
# 디렉토리 기본 경로 설정
DIR_PATH = path.join('..')
DIR_SAVE_PATH = path.join('.')

print(DIR_PATH)
print(DIR_SAVE_PATH)

..
.


In [5]:
# sql_bean.csv
bean_read = pd.read_csv(path.join('.', 'sql_bean.csv'), low_memory=False, encoding='cp949')

print(bean_read.shape)
bean_read.head()

(47, 8)


,idx,created_date,updated_date,name_ko,name_en,summary,thumbnail,user_grade
0,1,NaN,NaN,에티오피아 예가체프 G2 워시드 (중배전),에티오피아 예가체프 G2 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
1,2,NaN,NaN,과테말라 안티구아 SHB 워시드 (강배전),과테말라 안티구아 SHB 워시드 (강배전),아이템 요약 영역 입니다!!!,default_bean.png,0
2,3,NaN,NaN,에티오피아 코케 G1 펄프드내추럴 (중배전),에티오피아 코케 G1 펄프드내추럴 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
3,4,NaN,NaN,케냐 키암부 AA 워시드 (중배전),케냐 키암부 AA 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0
4,5,NaN,NaN,콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),콜롬비아 콜롬비아 SUPREMO 워시드 (중배전),아이템 요약 영역 입니다!!!,default_bean.png,0


In [6]:
# sql_member.csv
user_read = pd.read_csv(path.join('.', 'sql_member.csv'), low_memory=False, encoding='cp949')

print(user_read.shape)
user_read.head()

(1000, 12)


,idx,created_date,updated_date,age_range,expired,gender,hashcode,member_email,nickname,profileImg,role,sns_type
0,1,2023-03-01 12:00:00.000000,NaN,30~39,N,male,UADIR,jungjuon8355@gmail.com,정주온,http://k.kakaocdn.net/dn/dpk9l1/btqmGhA2lKL/Oz...,ROLE_MEMBER,KAKAO
1,2,2023-03-01 12:43:12.000000,NaN,30~39,N,male,7ILKE,pjc8001@naver.com,박종찬,http://k.kakaocdn.net/dn/dpk9l1/btqmGhA2lKL/Oz...,ROLE_MEMBER,KAKAO
2,3,2023-03-01 13:26:24.000000,NaN,20~29,N,male,Y4WZC,taehwan279@gmail.com,최태환,http://k.kakaocdn.net/dn/dpk9l1/btqmGhA2lKL/Oz...,ROLE_MEMBER,KAKAO
3,4,2023-03-01 14:09:36.000000,NaN,30~39,N,female,TK7SR,minyeon6103@naver.com,신민연,http://k.kakaocdn.net/dn/dpk9l1/btqmGhA2lKL/Oz...,ROLE_MEMBER,KAKAO
4,5,2023-03-01 14:52:48.000000,NaN,40~49,N,female,OHG9F,kwonyiyun8189@gmail.com,권예윤,http://k.kakaocdn.net/dn/dpk9l1/btqmGhA2lKL/Oz...,ROLE_MEMBER,KAKAO


In [7]:
bean_read.shape[0]

47

In [8]:
def generate_random_like(item_len):
    random_list = [random.choices([1, np.NaN], weights=[item_len * 0.2, item_len * 0.8])[0] for i in range(item_len)]
    random_series = pd.Series(random_list, index=[i+1 for i in range(item_len)])
    
    return random_series

In [9]:
generate_random_like(10)

1     NaN
2     1.0
3     NaN
4     NaN
5     NaN
6     NaN
7     1.0
8     NaN
9     NaN
10    NaN
dtype: float64

In [10]:
like_list_df = pd.DataFrame(index=user_read['idx'], columns=bean_read['idx'])
like_list_df.index.rename('member_idx', inplace=True)
like_list_df.columns.rename('bean_idx', inplace=True)
print(like_list_df.shape)
like_list_df.head()

(1000, 47)


bean_idx,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
member_idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
like_list_df = like_list_df.apply(lambda x: generate_random_like(like_list_df.shape[1]), axis=1)
like_list_df.columns.rename('item_idx', inplace=True)

In [12]:
like_list_df.head()

item_idx,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
member_idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
2,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN


In [13]:
like_list_df = like_list_df.melt(ignore_index=False).reset_index()
like_list_df.dropna(inplace=True)
like_list_df.reset_index(drop=True, inplace=True)
like_list_df.drop(['value'], axis=1, inplace=True)
like_list_df['created_date'] = ''
like_list_df['updated_date'] = ''
like_list_df['expired'] = 'N'
like_list_df['item_type'] = 'bean'
like_list_df = like_list_df[['created_date', 'updated_date', 'expired', 'item_idx', 'item_type', 'member_idx']]
like_list_df

,created_date,updated_date,expired,item_idx,item_type,member_idx
0,,,N,1,bean,7
1,,,N,1,bean,13
2,,,N,1,bean,15
3,,,N,1,bean,17
4,,,N,1,bean,21
...,...,...,...,...,...,...
9580,,,N,47,bean,987
9581,,,N,47,bean,988
9582,,,N,47,bean,992
9583,,,N,47,bean,996


In [14]:
# 파일 저장
os.makedirs(DIR_SAVE_PATH, exist_ok=True)
like_list_df.to_csv(path.join(DIR_SAVE_PATH, 'sql_like_list_new.csv'), sep=',', index=False, encoding='cp949')